In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as pt
import seaborn as sns
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [30]:
input_ads = pd.read_csv('train.csv')

#-----------------------------------------------------------------
print(input_ads.shape)
input_ads.head()


(15000, 19)


,id,Row#,clonesize,honeybee,bumbles,andrena,osmia,MaxOfUpperTRange,MinOfUpperTRange,AverageOfUpperTRange,MaxOfLowerTRange,MinOfLowerTRange,AverageOfLowerTRange,RainingDays,AverageRainingDays,fruitset,fruitmass,seeds,yield
0,0,639.0,25.0,0.50,0.25,0.75,0.63,94.6,57.2,79.0,68.2,33.0,55.9,34.0,0.56,0.402948,0.409261,31.274591,4418.44126
1,1,124.0,12.5,0.25,0.25,0.25,0.25,77.4,46.8,64.7,55.8,27.0,45.8,34.0,0.56,0.500438,0.445494,34.467567,5862.80545
2,2,485.0,25.0,0.50,0.25,0.38,0.75,94.6,57.2,79.0,68.2,33.0,55.9,24.0,0.39,0.509001,0.459421,36.624966,6079.08526
3,3,324.0,12.5,0.25,0.25,0.75,0.75,94.6,57.2,79.0,68.2,33.0,55.9,16.0,0.26,0.583379,0.498056,40.865478,7400.77538
4,4,235.0,12.5,0.25,0.25,0.50,0.63,77.4,46.8,64.7,55.8,27.0,45.8,24.0,0.39,0.447669,0.423764,33.298861,4858.24073


In [4]:
X = input_ads.drop(['yield', 'id', 'Row#'], axis=1)
y = input_ads['yield']

X, X_test, y, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

#--------------------------------------------------------------------------------
#Scaling the datasets
scaler = StandardScaler()

X_arr = scaler.fit_transform(X)
X_test_arr = scaler.fit_transform(X_test)

y_arr = np.array(y).reshape(X_arr.shape[0],1)
y_test_arr = np.array(y_test).reshape(X_test_arr.shape[0],1)

#--------------------------------------------------------------------------------
print(X_arr.shape)
print(X_test_arr.shape)
print(y_arr.shape)


(12000, 16)
(3000, 16)
(12000, 1)


In [31]:
from sklearn.tree import DecisionTreeRegressor

#---------------------------------------------------------------------------------------------------------
#Loss func
def loss_calc(y_true,y_pred):
    
    loss = np.sum(np.abs(y_true-y_pred))
        
    return loss

#---------------------------------------------------------------------------------------------------------
#Gradient Calc
def gradient_calc(y_true,y_pred):
    
    grad = -(y_true-y_pred)
    
    return grad

#---------------------------------------------------------------------------------------------------------
#The base estimator
def tree_creator(r_state,X,y):
    
    d_tree = DecisionTreeRegressor(random_state=r_state,criterion='absolute_error',
                                    max_depth=2,min_samples_split=5,
                                    min_samples_leaf=5,max_features=3)
    d_tree.fit(X,y)
    
    return d_tree

#---------------------------------------------------------------------------------------------------------
#Predicting through gradient boosting regression
def predict_grad_boost(models_tray,alpha,test_x=X_test_arr,train_y=y_arr):
    
    initial_pred = np.array([np.mean(train_y)] * len(test_x))
        
    final_pred = initial_pred.reshape(len(initial_pred),1)
    #print(final_pred.shape)
    
    for i in range(len(models_tray)):
        
        model = models_tray[i]
        temp_pred = (model.predict(test_x)).reshape(len(test_x),1)
        #print(temp_pred.shape)
        final_pred -= alpha * temp_pred
    
    return final_pred

In [32]:
def grad_boost_train(train_x,train_y,alpha=0.01,r_state=100,n_iters=101):

    model_tray = [] #Tray to collect the trained boosted stage estimators
    loss_counter = [] #Tray for loss capture

    
    initial_pred = np.array([np.mean(train_y)] * len(train_y))

    print('Initial val :',initial_pred.shape)
    model_pred = initial_pred.reshape(len(initial_pred),1)

    for epoch in range(n_iters): #Unit iteration

        if epoch%100==0:
            print('#---------- Epoch number :',epoch,' -----------#')
        
        #Calculating loss
        loss = loss_calc(y_true=train_y,
                         y_pred=model_pred)

        loss_counter.append(loss)
        
        #Calculating the gradient (residuals)
        grads = gradient_calc(y_true=train_y,
                              y_pred=model_pred)
        #print(grads.shape)
        #Building the regression tree on the gradient (residuals)
        tree_grad = tree_creator(r_state=r_state,
                                 X=train_x,
                                 y=grads)
        #print(train_x.shape)
        #print(tree_grad.predict(train_x).shape)
        
        #Predicting the residuals according to the tree fit above
        pred_m = (tree_grad.predict(train_x)).reshape(len(train_x),1)
        
        #Updating model through learning rate
        model_pred -= alpha * pred_m
        
        #Appending the model into tray
        model_tray.append(tree_grad)
        
    return model_tray,loss_counter,initial_pred


In [33]:
n_estimators = 1001 #No of boosting steps
alpha =0.01 #Learning rate

#Training gradient boosting regression
models_list,loss_counter,initial_pred = grad_boost_train(train_x=X_arr,
                                                         train_y=y_arr,
                                                         alpha=alpha,
                                                         r_state=100,
                                                         n_iters=n_estimators)


Initial val : (12000,)
#---------- Epoch number : 0  -----------#
#---------- Epoch number : 100  -----------#
#---------- Epoch number : 200  -----------#
#---------- Epoch number : 300  -----------#
#---------- Epoch number : 400  -----------#
#---------- Epoch number : 500  -----------#
#---------- Epoch number : 600  -----------#
#---------- Epoch number : 700  -----------#
#---------- Epoch number : 800  -----------#
#---------- Epoch number : 900  -----------#
#---------- Epoch number : 1000  -----------#


In [34]:
manual_gbm_pred = predict_grad_boost(models_tray=models_list, #Passing the fitted estimators into the predict function
                                     alpha=alpha, #The alpha val used during training
                                     test_x=X_test_arr) #Test dataset


In [35]:
from sklearn.metrics import mean_absolute_error

In [37]:
mean_absolute_error(y_test_arr,manual_gbm_pred)

635.9315321445847

In [38]:
from sklearn.ensemble import GradientBoostingRegressor

In [39]:

skl_gbm = GradientBoostingRegressor(random_state=100,n_estimators=1001,criterion='squared_error',
                                    max_depth=2,min_samples_split=5,
                                    min_samples_leaf=5,max_features=3)


In [40]:
skl_gbm.fit(X_arr,y_arr)

/home/aditya/.local/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(criterion='squared_error', max_depth=2,
                          max_features=3, min_samples_leaf=5,
                          min_samples_split=5, n_estimators=1001,
                          random_state=100)

In [41]:
skl_pred = skl_gbm.predict(X_test_arr)


In [42]:
mean_absolute_error(y_test_arr,skl_pred)

261.6564010870834

In [26]:
import numpy as np

class GradientBoostingRegressor:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.trees = []

    def _mse_split(self, X, y, feature_index, threshold):
        left_mask = X[:, feature_index] <= threshold
        right_mask = ~left_mask
        return left_mask, right_mask

    def _fit_tree(self, X, y):
        best_split = None
        best_mse = float('inf')
        best_left, best_right = None, None

        for feature_index in range(X.shape[1]):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                left_mask, right_mask = self._mse_split(X, y, feature_index, threshold)
                if len(y[left_mask]) == 0 or len(y[right_mask]) == 0:
                    continue

                left_error = np.var(y[left_mask]) * len(y[left_mask])
                right_error = np.var(y[right_mask]) * len(y[right_mask])
                mse = left_error + right_error

                if mse < best_mse:
                    best_mse = mse
                    best_split = (feature_index, threshold)
                    best_left, best_right = left_mask, right_mask

        return best_split, best_left, best_right

    def _build_tree(self, X, y, depth):
        if depth == self.max_depth or len(np.unique(y)) == 1:
            return np.mean(y)

        split, left_mask, right_mask = self._fit_tree(X, y)
        if split is None:
            return np.mean(y)

        left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)

        return (split, left_subtree, right_subtree)

    def fit(self, X, y):
        self.initial_prediction = np.mean(y)
        residuals = y - self.initial_prediction

        for _ in range(self.n_estimators):
            tree = self._build_tree(X, residuals, depth=0)
            self.trees.append(tree)

            predictions = self._predict_tree(tree, X)
            residuals -= self.learning_rate * predictions

    def _predict_tree(self, tree, X):
        if not isinstance(tree, tuple):
            return np.full(X.shape[0], tree)

        feature_index, threshold = tree[0]
        left_mask = X[:, feature_index] <= threshold
        right_mask = ~left_mask

        predictions = np.zeros(X.shape[0])
        predictions[left_mask] = self._predict_tree(tree[1], X[left_mask])
        predictions[right_mask] = self._predict_tree(tree[2], X[right_mask])

        return predictions

    def predict(self, X):
        predictions = np.full(X.shape[0], self.initial_prediction)

        for tree in self.trees:
            predictions += self.learning_rate * self._predict_tree(tree, X)

        return predictions


In [27]:
# Instantiate the model
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)

# Fit the model on the training data
gbr.fit(X_train, y_train)

# Predict on the test data
predictions = gbr.predict(X_test)

# Calculate the Mean Absolute Error (MAE)
mae = np.mean(np.abs(y_test - predictions))
print(f"Mean Absolute Error: {mae}")


NameError: name 'X_train' is not defined

In [2]:
df=pd.read_csv('train.csv')

In [3]:
X=df.drop(['yield', 'id', 'Row#'],axis=1)
y=df['yield']
from matplotlib import pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error
import pickle

In [4]:
class HigherOrderRegressor():
    def __init__(self, degree):
        self.degree = degree
        self.X = None
        self.Y = None
        self.B = None
        self.num_features = None

    def fit(self, X, Y):
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
        if len(Y.shape) == 1:
            Y = Y.reshape(-1, 1)
        self.num_features=X.shape[1]
        indices=np.random.shuffle(np.arange(X.shape[0]))
        temp_X = X[indices][0]
        temp_Y = Y[indices][0]
        # np.hstack((np.ones((self.X.shape[0], 1)), self.X))
        self.orig_X = np.hstack((np.ones((X.shape[0], 1)), temp_X))
        self.Y = temp_Y
        self.X = self.recursive_generate_powers(0, self.degree, np.ones(X.shape[0])).T
        try:
            self.B = np.matmul(np.linalg.pinv(np.matmul(self.X.T, self.X)),
                                np.matmul(self.X.T, self.Y))
        except:
            self.B = np.matmul(np.linalg.pinv(np.matmul(self.X.T, self.X)),
                                np.matmul(self.X.T, self.Y))

    def recursive_generate_powers(self, max_feature_encountered, current_degree, current_product, use_orig=True):
        if current_degree == 0:
            return current_product
        result=[]
        for i in range(max_feature_encountered, self.num_features+1):
            if use_orig:
                result.append(self.recursive_generate_powers(i, current_degree-1,
                                                            current_product*self.orig_X[:,i]))
            else:
                result.append(self.recursive_generate_powers(i, current_degree-1,
                                                            current_product*self.test_X[:,i], False))
        result=np.vstack(result)
        return result
    
    def predict(self, X):
        if X.ndim == 1:
            X = X.reshape(-1,1)
        self.test_X=np.hstack((np.ones((X.shape[0], 1)), X))
        self.final_testX=self.recursive_generate_powers(0, self.degree, np.ones(X.shape[0]), False).T
        return np.matmul(self.final_testX, self.B)

    def sum_of_squares(self, Y_true, Y_pred):
        return np.sum((Y_true - Y_pred)**2)
        
    def r2_score(self, Y_true, Y_pred):
        return 1 - np.sum((Y_true - Y_pred)**2) / np.sum((Y_true - np.mean(Y_true))**2)

    def k_fold_testing(self, k=10):
        width = self.X.shape[0]//k
        average_r2 = 0
        count = 0
        for i in range(0, self.X.shape[0], k):
            count += 1
            X_train = np.concatenate((self.X[:i], self.X[i+width:]))
            Y_train = np.concatenate((self.Y[:i], self.Y[i+width:]))
            X_test = self.X[i:i+width]
            Y_test = self.Y[i:i+width]
            try:
                B = np.matmul(np.linalg.pinv(np.matmul(X_train.T, X_train)), np.matmul(X_train.T, Y_train))
            except:
                B = np.matmul(np.linalg.pinv(np.matmul(X_train.T, X_train)), np.matmul(X_train.T, Y_train))
            predictions = np.matmul(X_test, B)
            average_r2 += self.r2_score(Y_test, predictions)
        # print(self.X.shape)
        try:
            B = np.matmul(np.linalg.pinv(np.matmul(self.X.T, self.X)), np.matmul(self.X.T, self.Y))
        except:
            B = np.matmul(np.linalg.pinv(np.matmul(self.X.T, self.X)), np.matmul(self.X.T, self.Y))
        return average_r2/count, np.matmul(self.X, B)
    
    def cross_validation(self, params):
        maximum_param=np.max(params)
        self.max_X=self.recursive_generate_powers(0, maximum_param, np.ones(self.X.shape[0])).T
        # print(self.max_X)
        responses = []
        factorials=[1]
        for i in range(1, maximum_param+self.num_features+1):
            factorials.append(factorials[-1]*i)
        for param in params:
            self.degree = param
            # print((factorials[param+self.num_features]//factorials[self.num_features])//factorials[param])
            self.X=self.max_X[:,:(factorials[param+self.num_features]//factorials[self.num_features])//factorials[param]]
            response, plot_values = self.k_fold_testing()
            responses.append(response)
            fig=plt.figure()
            plt.scatter(self.orig_X[:,1], self.Y)
            indices=np.argsort(self.orig_X[:,1])
            plt.plot(self.orig_X[:,1][indices], plot_values[indices], 'red')
            fig.savefig(f"degree_{param}.png")
        print(responses)
    
    def write_params(self):
        with open("3_weights.pkl", "wb") as f:
            pickle.dump(self.B, f)


In [5]:
reg=HigherOrderRegressor(1)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train=X_train.values
X_test=X_test.values
y_train=y_train.values
y_test=y_test.values

In [7]:
reg.fit(X_train, y_train)

In [8]:
mean_absolute_error(y_test, reg.predict(X_test))

269.74188837706777

In [10]:
errors=[]
for i in tqdm(range(1, 2**16)):
    mask = np.array(list(np.binary_repr(i, width=16)), dtype=int).astype(bool)
    X_train2=X_train[:,mask]
    X_test2=X_test[:,mask]
    X_train2=PolynomialFeatures(degree=1).fit_transform(X_train2)
    X_test2=PolynomialFeatures(degree=1).fit_transform(X_test2)
    reg.fit(X_train2, y_train)
    errors.append(mean_absolute_error(y_test, reg.predict(X_test2)))


100%|██████████| 65535/65535 [04:01<00:00, 271.36it/s]


In [9]:
from tqdm import tqdm

In [16]:
errors=[]
for i in tqdm(range(46390, 2**16)):
    mask = np.array(list(np.binary_repr(i, width=16)), dtype=int).astype(bool)
    X_train2=X_train[:,mask]
    X_test2=X_test[:,mask]
    X_train2=PolynomialFeatures(degree=2).fit_transform(X_train2)
    X_test2=PolynomialFeatures(degree=2).fit_transform(X_test2)
    reg.fit(X_train2, y_train)
    errors.append(mean_absolute_error(y_test, reg.predict(X_test2)))

100%|██████████| 19146/19146 [13:06<00:00, 24.33it/s]


In [14]:
print(np.min(errors))

265.4839192082926


In [15]:
print(np.argmin(errors))

5534


In [17]:
print(np.min(errors))

266.1416372492567


In [18]:
print(np.argmin(errors))

585


In [61]:
mask = np.array(list(np.binary_repr(5535, width=16)), dtype=int).astype(bool)
X_train2=X_train[:,mask]
X_test2=X_test[:,mask]
X_train2=PolynomialFeatures(degree=3).fit_transform(X_train2)
X_test2=PolynomialFeatures(degree=3).fit_transform(X_test2)
reg.fit(X_train2, y_train)
mean_absolute_error(y_test, reg.predict(X_test2))

262.75726659899703

In [48]:
test_data.columns

Index(['id', 'Row#', 'clonesize', 'honeybee', 'bumbles', 'andrena', 'osmia',
       'MaxOfUpperTRange', 'MinOfUpperTRange', 'AverageOfUpperTRange',
       'MaxOfLowerTRange', 'MinOfLowerTRange', 'AverageOfLowerTRange',
       'RainingDays', 'AverageRainingDays', 'fruitset', 'fruitmass', 'seeds',
       'yield'],
      dtype='object')

In [49]:
test_data=pd.read_csv('test.csv')

In [50]:
X_test_final=test_data.drop(['id', 'Row#'],axis=1)
X_test_final=X_test_final.values[:,mask]
X_test_final=PolynomialFeatures(degree=4).fit_transform(X_test_final)

In [52]:
test_data['yield']=reg.predict(X_test_final)

In [53]:
test_data_new=test_data[['id', 'yield']]

In [55]:
test_data_new.to_csv('hello.csv',index=False)

In [56]:
orig_preds=pd.read_csv('output.csv')['yield']
new_preds=pd.read_csv('hello.csv')['yield']

In [57]:
mean_absolute_error(orig_preds, new_preds)

97.11148719824867

In [1]:
%history -g -f second_file.py

In [3]:
import numpy as np
import pandas as pd
data=pd.read_csv('train.csv')
np.min(data['yield'])

1637.70402